# Train SALLY ensemble

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

from madminer.sampling import SampleAugmenter
from madminer.sampling import multiple_benchmark_thetas
from madminer.sampling import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.ml import MLForge, EnsembleForge

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [ ]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'

## Settings

In [4]:
n_estimators = 10

## Training function

In [5]:
def train_ensemble(filename, use_tight_cuts=True, n_estimators=10, **kwargs):
    cut_label = '_tight' if use_tight_cuts else ''
    
    ensemble = EnsembleForge(n_estimators, debug=False)

    ensemble.train_all(
        method='sally',
        x_filename=[sample_dir + 'train_local{}/x_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        t_xz0_filename=[sample_dir + 'train_local{}/t_xz_train_{}.npy'.format(cut_label, i) for i in range(n_estimators)],
        **kwargs
    )

    ensemble.calculate_expectation(
        x_filename=sample_dir + 'validation{}/x_validation.npy'.format(cut_label)
    )

    ensemble.save(model_dir + 'sally_ensemble_' + filename)

## All observables

In [ ]:
train_ensemble('all', use_tight_cuts=False)

In [ ]:
train_ensemble('all_tight', use_tight_cuts=True)

## Minimal observable basis

In [ ]:
train_ensemble(
    'minimal',
    use_tight_cuts=False,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

10:17  
10:17  ------------------------------------------------------------
10:17  |                                                          |
10:17  |  MadMiner v2018.11.06                                    |
10:17  |                                                          |
10:17  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
10:17  |                                                          |
10:17  ------------------------------------------------------------
10:17  
10:17  Training 10 estimators in ensemble
10:17  Training estimator 1 / 10 in ensemble
10:17  Starting training
10:17    Method:                 sally
10:17    Training data: x at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/x_train_0.npy
10:17                   t_xz (theta0) at  /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/train_local/t_xz_train_0.npy
10:17    Features:               [0, 5, 6, 7, 9, 10, 11]
10:17    Method:

12:10    Method:                 sally
12:10    Hidden layers:          (100, 100)
12:10    Activation function:    tanh
12:10    Batch size:             128
12:10    Trainer:                amsgrad
12:10    Epochs:                 50
12:10    Learning rate:          0.001 initially, decaying to 0.0001
12:10    Validation split:       0.25
12:10    Early stopping:         True
12:10    Scale inputs:           True
12:10    Regularization:         None
12:10  Loading training data
12:10  Found 1000000 samples with 2 parameters and 27 observables
12:10  Rescaling inputs
12:10  Only using 7 of 27 observables
12:10  Creating model for method sally
12:10  Training model
12:13    Epoch 5: train loss 27.9968 (mse_score: 27.9968)
12:13              val. loss  56.4557 (mse_score: 56.4557)
12:16    Epoch 10: train loss 27.9399 (mse_score: 27.9399)
12:16              val. loss  56.4850 (mse_score: 56.4850)
12:20    Epoch 15: train loss 27.8691 (mse_score: 27.8691)
12:20              val. loss  56

13:59              val. loss  15.2806 (mse_score: 15.2806)
14:02    Epoch 20: train loss 27.0021 (mse_score: 27.0021)
14:02              val. loss  15.3122 (mse_score: 15.3122)
14:06    Epoch 25: train loss 26.7801 (mse_score: 26.7801)
14:06              val. loss  15.2731 (mse_score: 15.2731)
14:10    Epoch 30: train loss 26.6030 (mse_score: 26.6030)
14:10              val. loss  15.3450 (mse_score: 15.3450)
14:14    Epoch 35: train loss 26.4305 (mse_score: 26.4305)
14:14              val. loss  15.3525 (mse_score: 15.3525)
14:17    Epoch 40: train loss 26.2876 (mse_score: 26.2876)
14:17              val. loss  15.3528 (mse_score: 15.3528)
14:21    Epoch 45: train loss 26.1920 (mse_score: 26.1920)
14:21              val. loss  15.3670 (mse_score: 15.3670)
14:24    Epoch 50: train loss 26.0938 (mse_score: 26.0938)
14:24              val. loss  15.3836 (mse_score: 15.3836)
14:24  Early stopping after epoch 11, with loss 15.23 compared to final loss 15.38
14:24  Finished training
14:24  

In [ ]:
train_ensemble(
    'minimal_tight',
    use_tight_cuts=True,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

In [ ]:
train_ensemble(
    'minimal_reg',
    use_tight_cuts=False,
    grad_x_regularization=0.1,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

In [ ]:
train_ensemble(
    'minimal_tight_reg',
    use_tight_cuts=True,
    grad_x_regularization=0.1,
    features=[[0,5,6,7,9,10,11] for _ in range(10)]
)

## Just resurrection phi

In [ ]:
train_ensemble(
    'resurrection',
    use_tight_cuts=True,
    features=[[26] for _ in range(10)]
)